In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date

In [2]:
result_list = []
for num in range(1, 530):
    url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL'
    params = {
        'page' : num
    }
    resp = requests.get(url, params = params)
    soup = BeautifulSoup(resp.content)
    tr_list = soup.find_all('tr')[1:]
    for tr in tr_list:
        time = tr.find_all('td')[0].text.strip()
        a = time.split('.')
        time2 = date(int(a[0]), int(a[1]), int(a[2]))
        price = tr.find_all('td')[1].text
        
        new_dict = {
            'date' : time2,
            'price' : price
        }
        result_list.append(new_dict)

In [3]:
df = pd.DataFrame(result_list)
df = df.set_index('date')
df.head()

,price
date,
2019-07-19,1.49
2019-07-18,1.50
2019-07-17,1.72
2019-07-16,1.72
2019-07-15,1.73


In [7]:
df.tail()

,price
date,
2004-08-26,3.52
2004-08-25,3.52
2004-08-24,3.52
2004-08-23,3.52
2004-08-20,3.46


In [8]:
import pandas as pd
from datetime import date, timedelta

In [9]:
date_data = pd.DataFrame(columns = ['date'])
start = date(2004, 8, 20)
end = date(2019, 7, 19)
for n in range(int((end - start).days) + 1):
    date_data.loc[n, 'date'] = start + timedelta(n)

In [10]:
date_data.head()

,date
0,2004-08-20
1,2004-08-21
2,2004-08-22
3,2004-08-23
4,2004-08-24


In [11]:
call = date_data.merge(df, on = 'date', how = 'outer')
call.head(10)

,date,price
0,2004-08-20,3.46
1,2004-08-21,NaN
2,2004-08-22,NaN
3,2004-08-23,3.52
4,2004-08-24,3.52
5,2004-08-25,3.52
6,2004-08-26,3.52
7,2004-08-27,3.51
8,2004-08-28,NaN
9,2004-08-29,NaN


In [12]:
call['price'] = call['price'].fillna(method='ffill')

In [13]:
call.head()

,date,price
0,2004-08-20,3.46
1,2004-08-21,3.46
2,2004-08-22,3.46
3,2004-08-23,3.52
4,2004-08-24,3.52


In [14]:
call = call.set_index('date')

In [15]:
call.head()

,price
date,
2004-08-20,3.46
2004-08-21,3.46
2004-08-22,3.46
2004-08-23,3.52
2004-08-24,3.52


In [16]:
def up_down_call(df):
    df_ud = pd.DataFrame(index = df.index, columns = ['up_down'])
    for i in range(len(df.index)):
        time = df.index[i]
        time2 = df.index[i] - pd.DateOffset(months = 1)
        
        if time.year >= 2005:

            while time2.date() not in df.index:
                time2 = time2 - pd.DateOffset(days = 1)
                
            if time2.date() in df.index:

                if df.loc[time].price > df.loc[time2.date()].price:
                    df_ud['up_down'][i] = '상승'
                elif df.loc[time].price == df.loc[time2.date()].price:
                    df_ud['up_down'][i] = 0
                else:
                    df_ud['up_down'][i] = '하락'
    return df_ud

In [17]:
M = up_down_call(call)
M.head()

,up_down
date,
2004-08-20,NaN
2004-08-21,NaN
2004-08-22,NaN
2004-08-23,NaN
2004-08-24,NaN


In [18]:
df_a = call.join(M, on = 'date', how = 'outer')
df_call = df_a.loc[date(2005, 1, 1):]

In [19]:
df_call

,price,up_down
date,,
2005-01-01,3.29,상승
2005-01-02,3.29,상승
2005-01-03,3.27,상승
2005-01-04,3.27,상승
2005-01-05,3.28,상승
2005-01-06,3.21,하락
2005-01-07,3.22,상승
2005-01-08,3.22,하락
2005-01-09,3.22,하락


In [20]:
df_call.to_excel('190723_morning_call.xlsx')

In [64]:
import os
os.getcwd()

'C:\\Users\\student\\Documents\\Daikoku_Python'

# 콜금리 상승/ 하락 판단  
 - 1개월전 금리와 비교하여 판단
 - 해당일이 존재하지 않을 경우 전날 가격으로 판단

### 판다스 DateOffset 모듈  
 - 알아서 다 해준다.

In [ ]:
day = date(2018, 1, 3)
a = day - pd.DateOffset(months = 1)
a

In [ ]:
df.loc[date(2005, 1, 3)]

In [ ]:
day = date(2005, 3, 31)
b = day - pd.DateOffset(months = 1)
print(b)
print(b.date())
df.loc[b.date()].price

In [ ]:
def up_down_call(df):
    df_ud = pd.DataFrame(index = df.index, columns = ['up_down'])
    for i in range(len(df.index)):
        time = df.index[i]
        time2 = df.index[i] - pd.DateOffset(months = 1)
        
        if time.year >= 2005:

            while time2.date() not in df.index:
                time2 = time2 - pd.DateOffset(days = 1)
                
            if time2.date() in df.index:

                if df.loc[time].price > df.loc[time2.date()].price:
                    df_ud['up_down'][i] = '상승'
                elif df.loc[time].price == df.loc[time2.date()].price:
                    df_ud['up_down'][i] = 0
                else:
                    df_ud['up_down'][i] = '하락'
    return df_ud

In [ ]:
df_t = up_down_call(df)
df_t.head()

In [ ]:
df_a = df.join(df_t, on = 'date', how = 'outer')
df_call = df_a.loc[:date(2005, 1, 1)]

In [ ]:
df_call.tail()

In [ ]:
type(df_call.index[1])

# 엑셀파이로 저장

In [ ]:
import os

In [ ]:
BASE_DIR = os.getcwd()
SAVE_DB_DIR = os.path.join(BASE_DIR, 'call')

if not os.path.exists(SAVE_DB_DIR):
    os.makedirs(SAVE_DB_DIR)

#텍스트 파일 만들기
temp_name = f"call_os.xlsx"
txt_name = os.path.join(SAVE_DB_DIR, temp_name)
excel_writer = pd.ExcelWriter(txt_name, engine = 'xlsxwriter')
#     df_report.to_excel(excel_writer, sheet_name='report')
df_call.to_excel(excel_writer)

excel_writer.save()

In [ ]:
df_call.to_excel('call_pd.xlsx')

In [ ]:
data_pd = pd.read_excel('call_pd.xlsx')

In [ ]:
data_pd.head()

In [ ]:
df_call.head()

In [ ]:
df_call.index[0]

In [ ]:
df_call.index[0] == data_pd.loc[0, 'date']

In [ ]:
data_pd.loc[0, 'date']

In [ ]:
data_pd['date'] = pd.to_datetime(data_pd['date']).apply(lambda x: x.date()).values

In [ ]:
data_pd.loc[0, 'date']

# 빈 날짜 채우기

In [ ]:
import pandas as pd
from datetime import date, timedelta

In [ ]:
date_data = pd.DataFrame(columns = ['date'])
start = date(2005, 1, 1)
end = date(2019, 7, 15)
for n in range(int((end - start).days) + 1):
    date_data.loc[n, 'date'] = start + timedelta(n)

In [ ]:
date_data.tail()

In [ ]:
date_data.loc[5308, 'date']

In [ ]:
data_dtype.loc[3, 'date']

In [ ]:
date_data.loc[5308, 'date'] == data_dtype.loc[3, 'date']

In [ ]:
import pandas as pd
import os

In [ ]:
cd call

In [ ]:
call = pd.read_excel('call_M.xlsx')

In [ ]:
call.head()

# timestamp형식을 timedelta로 바꾸기

In [ ]:
a = call.set_index('date')
a.index[1]

In [ ]:
type(date_data.loc[10, 'date'])

In [ ]:
type(call.loc[10, 'date'])

In [ ]:
call['date'] = pd.to_datetime(call['date']).apply(lambda x: x.date()).values

In [ ]:
import pandas as pd

In [ ]:
type(call.loc[10, 'date'])

In [ ]:
call.head()

# 두 데이터 합치기

In [ ]:
M = date_data.merge(call, on = 'date', how = 'outer')
M.head(10)

In [ ]:
M['price'] = M['price'].fillna(method='bfill')

In [ ]:
M.head()

In [ ]:
M.to_excel('콜금리만 채우기.xlsx')

In [ ]:
M['up_down'] = M['up_down'].fillna(method='bfill')

In [ ]:
M.head()

In [ ]:
M.to_excel('up_down도 채우기.xlsx')